Fernanda Farel R
HCK-005

# Home Credit Score Card


## Problem

Home Credit Indonesia wants to create Machine Learning model to help the marketing team determine whether the proposed loan is accepted or gottin rejected. Then after analyze several a data, the team wants to find out what customer behaviour that often make a loans are accepted to determine the next trends.

In [ ]:
pip install feature_engine


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from feature_engine.outliers import Winsorizer

# Evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score

# Display
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, accuracy_score, f1_score, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline

# model
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler


In [ ]:
df = pd.read_csv('application_train.csv')
df.head(10)

Getting insight from the data

The number of customer who apply for loans

In [ ]:
num1 = df.groupby('TARGET')['CODE_GENDER'].count().reset_index()
num1.rename(columns = {'CODE_GENDER' : 'Customers'}, inplace=True)

num1['C_percentage'] = round(num1.Customers / num1.Customers.sum(), 2)
num1


From the table above, there is 92% did not have any issues in repaying the loan in given the time and 8% was have a issues. So lets take a look into the criteria whether the customer not have any issues in repaying the loan

# Marital Status

In [ ]:
df.NAME_FAMILY_STATUS.value_counts()

there is 6 marital category, lets make it simple into just 2 categories, namely married and single

- Married = people who's not single
- Single = people who's not married

In [ ]:
Marital_status = []
married = ['Married', 'Civil_marriage']

for i in df.NAME_FAMILY_STATUS:
  if i in married:
    Marital_status.append('Married')
  else:
    Marital_status.append('Single')

df['Marital_status'] = Marital_status
df.drop(columns='NAME_FAMILY_STATUS', inplace=True)

In [ ]:
ms = df[df['TARGET'] == 0].groupby(['Marital_status'])['CODE_GENDER'].count().reset_index()
ms.rename(columns = {'CODE_GENDER' : 'Customers'}, inplace=True)

ms['C_percentage'] = round(num1.Customers / num1.Customers.sum(), 2)
ms

56% from customer didn't have any issues to pay the loan is married category.



In [ ]:
ms1 = df[df['TARGET'] == 1].groupby(['Marital_status'])['CODE_GENDER'].count().reset_index()
ms1.rename(columns = {'CODE_GENDER' : 'Customers'}, inplace=True)

ms1['C_percentage'] = round(ms1.Customers / ms1.Customers.sum(), 2)
ms1

In [ ]:
ms2 = df[df['Marital_status'] == 'Single'].groupby(['TARGET', 'Marital_status'])['CODE_GENDER'].count().reset_index()
ms2.rename(columns = {'CODE_GENDER' : 'Customers'}, inplace=True)

ms2['C_percentage'] =  round(ms2.Customers / ms2.Customers.sum(), 2)
ms2

In [ ]:
ms3 = df[df['Marital_status'] == 'Married'].groupby(['TARGET', 'Marital_status'])['CODE_GENDER'].count().reset_index()
ms3.rename(columns = {'CODE_GENDER' : 'Customers'}, inplace=True)

ms3['C_percentage'] = round(ms3.Customers / ms3.Customers.sum(), 2)
ms3

# Contract type

In [ ]:
ctype = df.groupby(['TARGET', 'NAME_CONTRACT_TYPE'])['CODE_GENDER'].count().reset_index().sort_values(by = 'NAME_CONTRACT_TYPE')
ctype.rename(columns = {'CODE_GENDER' : 'Customers'}, inplace=True)

name_of_ctype = list(ctype['NAME_CONTRACT_TYPE'].unique())
percentage = []

for i in name_of_ctype:
  data = ctype[ctype['NAME_CONTRACT_TYPE'] == i]
  total = data['Customers'].sum()
  for x, y in enumerate(data['Customers']):
    pct = round(y/total, 2)
    percentage.append(pct)

ctype['Percentage'] = percentage
ctype

- Loan Distribution: Out of the total customers, 92% have cash loans, while only 8% have revolving loans.

- Repayment Status: Among customers with cash loans, 92% have repaid the loan (TARGET=0), while 8% defaulted (TARGET=1). For customers with revolving loans, 95% have repaid, and 5% defaulted.

- Loan Popularity: Cash loans appear to be more popular among customers, with a higher count and percentage compared to revolving loans.

# Occupation type

In [ ]:
octype = df.groupby(['TARGET', 'OCCUPATION_TYPE'])['CODE_GENDER'].count().reset_index().sort_values(by=['OCCUPATION_TYPE', 'TARGET'])
octype.rename(columns = {'CODE_GENDER' : 'Customers'}, inplace=True)

name_of_ot = list(octype['OCCUPATION_TYPE'].unique())
percentage = []

for i in name_of_ot:
    data = octype[octype['OCCUPATION_TYPE'] == i]
    total = data['Customers'].sum()
    for x,y in enumerate(data['Customers']):
        pct = round(y / total, 2)
        percentage.append(pct)

octype['percentage'] = percentage
octype

Based on these percentages, the occupation type with the highest percentage is "Secretaries" with a percentage of 0.97 for TARGET=0, indicating that 97% of secretaries have repaid their loans.

In [ ]:
octype2 = octype[octype['OCCUPATION_TYPE'].isin(['Accountants', 'Core staff', 'Secretaries'])]
octype2 = octype2.reset_index()
octype2.drop(columns = 'index', inplace=True)

mapping_target = {
    1 : 'Have Issues',
    0 : 'Dont Have Issues'
}

octype2['TARGET'] = octype2['TARGET'].map(mapping_target)


ax = sns.barplot(x = 'OCCUPATION_TYPE', y= 'Customers', hue= 'TARGET', data = octype2)
plt.xlabel('Job of Customer')
plt.ylabel('Total Customer')
plt.legend(title = 'Status', bbox_to_anchor = (1.45,1))

for i, v in enumerate(octype2['Customers']):
    if i%2 == 0:
        plt.text(i/2 - 0.35, v + 100, s= str(octype2['percentage'][i]* 100) + '%', color='black', fontweight='bold')
    else:
        plt.text(i/2 - 0.45, v + 100, s= str(octype2['percentage'][i] * 100) + '%', color='black', fontweight='bold')

plt.show(ax)

# Age of the Customers

In [ ]:
df['Age'] = np.floor(df['DAYS_BIRTH'] / -365).astype(int)
df.drop(columns= 'DAYS_BIRTH', inplace=True)

In [ ]:
age_cat = []
for i in df['Age']:
    if i < 36:
        age_cat.append('Young Adult')
    elif i < 51:
        age_cat.append('Adult')
    else:
        age_cat.append('Older')

df['Age_category'] = age_cat

In [ ]:
g_age = df.groupby(['TARGET', 'Age_category'])['CODE_GENDER'].count().reset_index().sort_values('Age_category').reset_index()
g_age.rename(columns = {'CODE_GENDER' : 'Customers'}, inplace=True)
g_age.drop(columns = 'index', inplace=True)

name_of_ot = list(g_age['Age_category'].unique())
percentage = []

for i in name_of_ot:
    data = g_age[g_age['Age_category'] == i]
    total = data['Customers'].sum()
    for x,y in enumerate(data['Customers']):
        pct = round(y / total, 2)
        percentage.append(pct)

g_age['percentage'] = percentage

mapping_target = {
    1 : 'Have Issues',
    0 : 'Dont Have Issues'
}

g_age['TARGET'] = g_age['TARGET'].map(mapping_target)
g_age

In [ ]:
ax = sns.barplot(x = 'Age_category', y= 'Customers', hue= g_age.TARGET, data = g_age)
plt.legend(title = 'Status', bbox_to_anchor = (1.45,1))

for i, v in enumerate(g_age['Customers']):
    if i%2 == 0:
        plt.text(i/2 - 0.35, v/2, s= str(g_age['percentage'][i]* 100) + '%', color='black', fontweight='bold')
    else:
        plt.text(i/2 - 0.45, v + 2000, s= str(g_age['percentage'][i] * 100) + '%', color='black', fontweight='bold')

plt.show(ax)

In [ ]:
df.drop(columns = 'Age_category', inplace=True)

In [ ]:
itype = df.groupby(['TARGET', 'NAME_INCOME_TYPE', 'NAME_CONTRACT_TYPE'])['CODE_GENDER'].count().reset_index().sort_values(['NAME_INCOME_TYPE', 'NAME_CONTRACT_TYPE']).reset_index()
itype.rename(columns = {'CODE_GENDER' : 'Customers'}, inplace=True)
itype.drop(columns = 'index', inplace=True)

income_type   = list(itype['NAME_INCOME_TYPE'].unique())
percentage  = []

for i in income_type:
    data = itype[itype.NAME_INCOME_TYPE == i]
    contract_type = list(data['NAME_CONTRACT_TYPE'].unique())
    for x in contract_type:
        data2 = data[data.NAME_CONTRACT_TYPE == x]
        total = data2['Customers'].sum()
        for y,z in enumerate(data2['Customers']):
            pct = round(z / total, 2)
            percentage.append(pct)

itype['percentage'] = percentage
itype

In [ ]:
itype_rl = itype[itype['NAME_CONTRACT_TYPE'] == 'Revolving loans'].reset_index()

mapping_target = {
    1 : 'Have Problem',
    0 : 'Dont Have Problem'
}

itype_rl['TARGET'] = itype_rl['TARGET'].map(mapping_target)

ax = sns.barplot(x = 'NAME_INCOME_TYPE', y= 'Customers', data = itype_rl, hue = 'TARGET')
plt.title('Contract Type = Revolving Loans')
plt.xticks(rotation=45)
plt.legend(title = 'Status', bbox_to_anchor = (1.45,1))

#for i, v in enumerate(itype_rl['Customers']):
 #   plt.text(i/2 - 0.2, v + 200, s= str(itype_rl['percentage'][i]* 100) + '%', color='black', fontweight='bold', fontsize = 8)
i = 0
itype_p = itype_rl.sort_values('TARGET').reset_index()
for p in ax.patches:
    widht, height = p.get_x(), p.get_height()
    if np.isnan(height) == True:
        continue
    else:
        plt.text(widht, height + 200, s = str(itype_p['percentage'][i]* 100) + '%' )
        i = i +1

plt.show()

In [ ]:
itype_cl = itype[itype['NAME_CONTRACT_TYPE'] == 'Cash loans'].reset_index()

mapping_target = {
    1 : 'Have Issues',
    0 : 'Dont Have Issues'
}

itype_cl['TARGET'] = itype_cl['TARGET'].map(mapping_target)

ax = sns.barplot(x = 'NAME_INCOME_TYPE', y= 'Customers', data = itype_cl, hue = 'TARGET')
plt.title('Contract Type = Cash Loans')
plt.xticks(rotation=45)
plt.legend(title = 'Status', bbox_to_anchor = (1.45,1))

#for i, v in enumerate(itype_rl['Customers']):
 #   plt.text(i/2 - 0.2, v + 200, s= str(itype_rl['percentage'][i]* 100) + '%', color='black', fontweight='bold', fontsize = 8)
i = 0
itype_p2 = itype_cl.sort_values('TARGET').reset_index()
for p in ax.patches:
    widht, height = p.get_x(), p.get_height()
    if np.isnan(height) == True:
        continue
    else:
        plt.text(widht, height + 200, s = str(itype_p2['percentage'][i]* 100) + '%' )
        i = i +1

plt.show()


In [ ]:
itype_cl

# Data Cleaning

Detecting and remove data duplicated

In [ ]:
df.duplicated().sum()

In [ ]:
count_nv = pd.DataFrame(df.isnull().sum(), columns=['Total Null Data']).reset_index()
count_nv['Percentage'] = ( count_nv['Total Null Data'] / len(df) ) * 100
count_nv[count_nv['Percentage'] > 40].sort_values(by='Percentage', ascending=False)

In [ ]:
data = list(count_nv[count_nv['Percentage'] > 40]['index'])
df = df.drop(columns = data)
df = df.drop(columns = 'SK_ID_CURR')

In [ ]:
count_nv = pd.DataFrame(df.isnull().sum(), columns=['Total Null Data']).reset_index()
count_nv['Percentage'] = ( count_nv['Total Null Data'] / len(df) ) * 100
still_null = count_nv[count_nv['Percentage'] != 0].sort_values(by='Percentage', ascending=False)

In [ ]:
a = list(still_null['index'])
df[a].dtypes

for i in a:
    data = df[i]
    types = str(data.dtype)
    if types == 'object':
        df[i].fillna(df[i].mode()[0], inplace=True)
    else:
        df[i].fillna(df[i].median(), inplace=True)

In [ ]:
count_nv = pd.DataFrame(df.isnull().sum(), columns=['Total Null Data']).reset_index()
count_nv['Percentage'] = ( count_nv['Total Null Data'] / len(df) ) * 100
count_nv[count_nv['Percentage'] > 0].sort_values(by='Percentage', ascending=False)

# Replace unknown value

In [ ]:
df['CODE_GENDER'] = df['CODE_GENDER'].replace(['XNA'], df['CODE_GENDER'].mode()[0])
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace(['XNA'], df['ORGANIZATION_TYPE'].mode()[0])

# Remove flag documents

In [ ]:
FLAG_DOCUMENT = [col for col in df.columns if 'FLAG_DOCUMENT' in col]
df.drop(columns = FLAG_DOCUMENT, axis=1, inplace=True)

# Feature Engineering

In [ ]:
x = df.drop(columns = 'TARGET')

# Num dtype

In [ ]:
num = x.select_dtypes(exclude='object')

In [ ]:
num.head()

# Change neg value on columns to pos value

In [ ]:
neg_column = num.columns[(num < 0).any()].tolist()
num[neg_column] = num[neg_column] * -1
num.head()

# Look for a numeric data type that has a unique value > 2

In [ ]:
df_num = pd.DataFrame(num.nunique(), columns = ['Total_unique']).reset_index()
df_num = df_num.rename(columns = {'index' : 'Name_of_column'})

num_a = []
num_b = []
for i,y in enumerate(df_num.Name_of_column):
    if df_num['Total_unique'][i] > 2:
        num_a.append(y)
    else:
        num_b.append(y)

# Data Distribution

In [ ]:
plt.figure(figsize=(20,12))

for i, column in enumerate (num[num_a].columns, 1):
    plt.subplot(4,7,i)
    sns.kdeplot(data=num[num_a], x=num[column])
    plt.tight_layout()

In [ ]:
plt.figure(figsize=(20,12))

for i, column in enumerate (num[num_a].columns, 1):
    plt.subplot(7,4,i)
    sns.boxplot(data=num[num_a], x=num[column])
    plt.tight_layout()

# Remove and handling outlier

In [ ]:
print(f'Row before delete outlier at CNT_children :  {len(df)}')

filtered_entries = np.array([True] * len(df))

zscore = abs(stats.zscore(df['CNT_CHILDREN']))
filtered_entries = (zscore < 3) & filtered_entries

df = df[filtered_entries]

print(f'Row after delete outlier at CNT_children : {len(df)}')

# Feature Transformation

In [ ]:
x = df.drop(columns = 'TARGET', axis=1)
num = x[num_a]

In [ ]:
num_mean = num.mean()
num_std = num.std()

num = (num - num_mean) / num_std
num.head()

In [ ]:
num_1 = x[num_b]
num_1 = num_1.loc[~num_1.index.duplicated(keep='first')]

num = pd.concat([num, num_1], axis=1)
num

# Feature Transformation

In [ ]:
obj = x.select_dtypes(include='object')
df_obj = pd.DataFrame(obj.nunique(), columns = ['Total_unique']).reset_index()
df_obj = df_obj.rename(columns = {'index' : 'Name_of_column'})

obj_1 = []
obj_2 = []
for i,y in enumerate(df_obj.Name_of_column):
    if df_obj['Total_unique'][i] > 2:
        obj_1.append(y)
    else:
        obj_2.append(y)
obj


# One Hot Encoder

In [ ]:
for cat in obj_1:
    onehots = pd.get_dummies(obj[cat], prefix=cat)
    obj = obj.join(onehots)

obj = obj.drop(columns = obj_1)

# Label Encoder

In [ ]:
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
for cat in obj_2:
    obj[cat]= label_encoder.fit_transform(obj[cat])

obj

In [ ]:
df_train = pd.concat([num, obj], axis=1)

In [ ]:
df = pd.concat([df_train, df['TARGET']],axis=1)

# Data Splitting

In [ ]:
x = df.drop(columns = 'TARGET')
y = df['TARGET']

# Oversampling

In [ ]:
sm = SMOTE(random_state = 2)
x_over, y_over = sm.fit_resample(x, y.ravel())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_over, y_over, test_size=0.3, random_state=42)

# Undersampling

In [ ]:
rus = RandomUnderSampler(random_state=42)
x_under, y_under = rus.fit_resample(x, y.ravel())

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(x_under, y_under, test_size=0.3, random_state=42)


# Train logistic regression

In [ ]:
def confusionmatrix(predictions, y_test):
    cm = confusion_matrix(y_test, predictions)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    return disp.plot()

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train.ravel())
predictions = logreg.predict(X_test)

# print classification report
print(classification_report(y_test, predictions))
confusionmatrix(predictions, y_test)

# Undersampling data

In [ ]:
logreg1 = LogisticRegression()
logreg1.fit(X_train2, y_train2)
predictions2 = logreg1.predict(X_test2)

# print classification report
print(classification_report(y_test2, predictions2))
confusionmatrix(predictions2, y_test2)

In [ ]:
predictions = logreg1.predict(X_train2)

# print classification report
print(classification_report(y_train2, predictions))
confusionmatrix(predictions, y_train2)

# XGBoost model training

In [ ]:
xgb = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb.fit(X_train, y_train.ravel())

predictions = xgb.predict(X_test)

# print classification report
print(classification_report(y_test, predictions))
confusionmatrix(predictions, y_test)

In [ ]:
predictions = xgb.predict(X_train)

# print classification report
print(classification_report(y_train, predictions))
confusionmatrix(predictions, y_train)

# Random Forest Modelling

In [ ]:
rforest = RandomForestClassifier().fit(X_train, y_train)
predictions = rforest.predict(X_test)

# print classification report
print(classification_report(y_test, predictions))
confusionmatrix(predictions, y_test)

In [ ]:
predictions = rforest.predict(X_train)

# print classification report
print(classification_report(y_train, predictions))
confusionmatrix(predictions, y_train)

# Prediction

In [ ]:
df_test = pd.read_csv('application_test.csv')
df_test.head()

In [ ]:
df_test['CODE_GENDER'] = df_test['CODE_GENDER'].replace(['XNA'], df_test['CODE_GENDER'].mode()[0])
df_test['ORGANIZATION_TYPE'] = df_test['ORGANIZATION_TYPE'].replace(['XNA'], df_test['ORGANIZATION_TYPE'].mode()[0])

Marital_status = []
married = ['Married', 'Single']

for i in df_test.NAME_FAMILY_STATUS:
    if i in married:
        Marital_status.append('Married')
    else:
        Marital_status.append('Single')

df_test['Marital_status'] = Marital_status
df_test.drop(columns= 'NAME_FAMILY_STATUS', inplace=True)

df_test['Age'] = np.floor(df_test['DAYS_BIRTH'] / -365).astype(int)
df_test.drop(columns= 'DAYS_BIRTH', inplace=True)

In [ ]:
count_nv = pd.DataFrame(df_test.isnull().sum(), columns=['Total Null Data']).reset_index()
count_nv['Percentage'] = ( count_nv['Total Null Data'] / len(df) ) * 100
still_null = count_nv[count_nv['Percentage'] > 0].sort_values(by='Percentage', ascending=False)

In [ ]:
a = list(still_null['index'])

for i in a:
    data = df_test[i]
    types = str(data.dtype)
    if types == 'object':
        df_test[i].fillna(df_test[i].mode()[0], inplace=True)
    else:
        df_test[i].fillna(df_test[i].median(), inplace=True)

In [ ]:
num = df_test[num_a]

In [ ]:
num = (num - num_mean) / num_std
num.head()

In [ ]:
num_1 = df_test[num_b]
num_1 = num_1.loc[~num_1.index.duplicated(keep='first')]

num = pd.concat([num, num_1], axis=1)
num

In [ ]:
obj = df_test.select_dtypes(include='object')

for cat in obj_1:
    onehots = pd.get_dummies(obj[cat], prefix=cat)
    obj = obj.join(onehots)

obj = obj.drop(columns = obj_1)

In [ ]:
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
for cat in obj_2:
    obj[cat]= label_encoder.fit_transform(obj[cat])

obj

In [ ]:
df_test = pd.concat([num, obj], axis=1)

In [ ]:
train = set(df.columns)

In [ ]:
train.remove('TARGET')

In [ ]:
not_in_test = train - set(df_test.columns)
not_in_test = list(not_in_test)

In [ ]:
df_test[not_in_test] = 0

In [ ]:
column_train = list(df.columns)
column_train.remove('TARGET')
df_test = df_test[column_train]

In [ ]:
predictions = rforest.predict(df_test)

In [ ]:
df_test['TARGET'] = predictions

In [ ]:
df_test['TARGET'].value_counts()